In [ ]:
######################################################################################
# Using a Policy Gradient Model to beat the CPU at the PLE pong environment
# Implementing the REINFORCE algorithm (Monte Carlo policy gradient)

# Need to: 
# - Implement policy gradient -> DONE
# - Add reward-to-go -> DONE
# - Add discounting -> DONE
# - Add baseline
# - Add a replay buffer

# Karphathy's took 30 hours of training & 10,000 episodes
# Karpathy used a larger layer 200, lr 1e-4, batch was done over 10 episodes

######################################################################################

In [ ]:
from ple import PLE
import pygame
import numpy as np
import import_ipynb
from ple.games.catcher import Catcher
import pickle

from Policy_Gradient_Model import *
from Utils import *
from Game import Pong
from Back_up_Policy import *

##############################
# VARIABLES 
RENDER = True
MAX_SCORE = 21
NB_FRAMES = 10000
EPISODES = 10000
UPDATE_FREQ = 5
##############################   

gamma = 0.99
lr = 0.01
        
policy_model = agent(lr, gamma)

reward_tally = []
time_tally = []
reward_av = []

rewards = []
states = []
actions = []

######################################################################################

for e in range(1, EPISODES + 1):

    print("Starting Episode {}".format(e))

    pygame.init()
    game = Catcher(width=256, height=200)
    # game = Pong(width=256, height=200, MAX_SCORE=MAX_SCORE)
    game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)
    env = PLE(game, fps=30, 
              display_screen=RENDER, 
              force_fps=True,
              rng=np.random.RandomState(1),
              reward_values={'tick': 0.0, 'win': 0.0, 'loss': 0.0, 'positive': 1.0, 'negative': -1.0}
             )
    env.init()

    # get the possible actions
    possible_actions = game.actions 
    # action_names = ['up', 'down']
    action_names = ['left', 'right']

    state = process_state(env.getGameState())
    total_reward = 0

    # for some reason breaks without this    
    env.act(env.NOOP)
    reward = 0.0

    for f in range(NB_FRAMES):           

        action = policy_model.act(state)        
        reward = env.act(possible_actions[action_names[action]])
        next_state = process_state(env.getGameState())         

        rewards.append(reward)
        states.append(state)
        actions.append(action)

        state = next_state
        total_reward += reward

        #check if the game is over
        if env.game_over():            
            pygame.quit()
            
            if e % UPDATE_FREQ == 0:                
                policy_model.train(states, rewards, actions)
                rewards = []
                states = []
                actions = []
            
            print("total reward after {} episodes is {}".format(e, total_reward))
            reward_tally.append(total_reward)
            time_tally.append(f)

            """
            if e == 1:
                policy_model.load_model('model_pong_2.h5')                
            """    
            break      

    # Save the model  
    if e % 50 == 0:
        policy_model.save_model('model_pong_2.h5')                

    # save the results
    if e % 100 == 0:        
        reward_av.append([np.mean(reward_tally), np.std(reward_tally), np.mean(time_tally)])
        reward_tally = []
        time_tally = []
        with open("./Rewards_Files/rewards-ep-" + str(e) + ".txt", "wb") as file:
            pickle.dump(reward_av, file)                    

    ###########################################################################


In [2]:
import matplotlib.pyplot as plt
import pickle

with open("./Rewards_Files/rewards-ep-500.txt", "rb") as fp1:   # Unpickling
    b = pickle.load(fp1)
    
print(b)

"""
x = range(1,51)
y1 = b[0] 
y2 = b[1]

plt.plot(x,y1)
plt.ylabel('mean_rewards')
plt.ylim(-45, -50)
plt.show()
"""

[[-1.28, 1.3717142559585798, 185.49], [-1.33, 1.6003437130816618, 183.63], [-1.36, 1.5001333274079343, 182.29], [-1.45, 1.6874537030686205, 179.59], [-1.45, 1.6575584454250776, 179.68]]


"\nx = range(1,51)\ny1 = b[0] \ny2 = b[1]\n\nplt.plot(x,y1)\nplt.ylabel('mean_rewards')\nplt.ylim(-45, -50)\nplt.show()\n"